*__Note:__* Filepaths should be changed as necessary

In [1]:
import os
from pathlib import Path
import csv
from collections import defaultdict
import json
import numpy as np
import pandas as pd

In [2]:
cwd = os.getcwd()
print(cwd)
studiesDirPath = "/home/NIDDK/DATA/TEDDY/"
dataDictionaryJSON = cwd + "/TEDDY_data_dictionary_v1.2.json";
studyCSVFiles = (file for file in os.listdir(studiesDirPath) 
         if os.path.isfile(os.path.join(studiesDirPath, file)))

In [4]:
studyFileNamesMap = {}
for filename in studyCSVFiles:
    studyFile = Path(filename).stem
    studyFileNameLowerCase = studyFile.lower()
    studyFileNamesMap[studyFileNameLowerCase] = filename

In [5]:
#Check for data stype and max length anomolies
studyFileColumnMap = {}     
list_of_column_names = []

In [ ]:
#Read Data Dictionary
with open(dataDictionaryJSON) as json_data:
    dataDictionary = json.load(json_data)
    allTypes = set()
    for eachEntityrow in dataDictionary:
        if (not eachEntityrow.startswith("teddy") and not eachEntityrow.startswith("test_results")):
        #print("Processing entity: " + eachEntityrow)
            dictionaryValues = dataDictionary[eachEntityrow]
        if (studyFileNamesMap.get(eachEntityrow)):
            studyFileName = studyFileNamesMap.get(eachEntityrow)
            dictColumnNames = list()
            for issue in dictionaryValues:
                dictColumnNames.append(issue['Variable'].lower())
            #Parse the corresponding data file
            with open(studiesDirPath + studyFileName, mode='r', encoding='utf-8') as f:
                reader = csv.DictReader(f, delimiter=',')
                columnNamesInStudyFile = list()
                for col in reader.fieldnames:           
                    columnNamesInStudyFile.append(col)
                maxLengthExceededColumns = {}
                typeCheckAnolomies = {}
                noDataTypeDefinitions = {}
                maxLengthExceededColumns["Table ~ Column ~ Row Num"] = "Value ~ Max Length from Dict ~ Value Length"
                typeCheckAnolomies["Table ~ Column ~ Row Num"] = "Value"
                noDataTypeDefinitions["Table ~ Column ~ Row Num"] = "Value"
                rowNum = 0
                for row in reader:
                    rowNum = rowNum + 1
                    for columnName in columnNamesInStudyFile:
                        columnValue = row[columnName]
                        if (columnName.lower() in dictColumnNames):
                            for issue in dictionaryValues:
                                if (issue['Variable'].lower() == columnName.lower()):
                                    dictCol = issue['Type']
                                    maxLength = issue['Len']
                                    if (dictCol == 'Num'):
                                        if (columnValue != "" and maxLength != ""):
                                            roundedVal = round(float(columnValue))
                                            if (len(str(roundedVal)) > int(maxLength)):
                                                if columnName not in maxLengthExceededColumns:
                                                    maxLengthExceededColumns[eachEntityrow + "~" + columnName + "~" + str(rowNum)] = str(columnValue) + " ~ " +  str(maxLength) + " ~ " +  str(roundedVal)
                                    else: 
                                        if (columnValue != "" and maxLength != "" and (len(str(columnValue)) > float(int(maxLength)))):
                                            if columnName not in maxLengthExceededColumns:
                                                maxLengthExceededColumns[eachEntityrow + "~" + columnName + "~" + str(rowNum)] = str(columnValue) + " ~ " +  str(maxLength) + " ~ " +  str(roundedVal)
                                    if dictCol not in allTypes:
                                        allTypes.add(dictCol)
                                    if (dictCol == 'Num'):
                                        numbercheck = columnValue.isnumeric()
                                        decimalCheck = isinstance(columnValue, float)
                                        if (not((numbercheck == True and decimalCheck == False) or (numbercheck == False or decimalCheck == True))):
                                            if columnName not in typeCheckAnolomies:
                                                typeCheckAnolomies[eachEntityrow + "~" + columnName + "~" + str(rowNum)] = columnValue
                                    if (dictCol == 'Char'):
                                        charCheck = isinstance(columnValue, str)
                                        if (charCheck == False):
                                            if columnName not in typeCheckAnolomies:
                                                typeCheckAnolomies[eachEntityrow + "~" + columnName + "~" + str(rowNum)] = columnValue                               
                if (len(maxLengthExceededColumns) > 1):
                    df = pd.DataFrame(maxLengthExceededColumns, index=[0])
                    print("Max Length Exceeded:")
                    print(df)
                if (len(typeCheckAnolomies) > 1):
                    df1 = pd.DataFrame(typeCheckAnolomies, index=[0])
                    print("Data Type Anomalies:")
                    print(df1)
                if (len(noDataTypeDefinitions) > 1):
                    df2 = pd.DataFrame(noDataTypeDefinitions, index=[0])
                    print("No data type defined for:")
                    print(df2) 
        else:
            print("File not found:" + eachEntityrow)